<a href="https://colab.research.google.com/github/CASAttackZW2025/CAS502Project/blob/main/CAS502IngestionV001.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# Function: create_dummy_df, I need to creae a test data frame for each sheet
def create_dummy_df(num_rows, columns):
    """
    Create a dummy DataFrame with the given number of rows and columns.
    Each cell in a column is filled with a string in the format 'ColumnName_rowIndex'.
    This function is used to generate test data for testing purposes.
    """
    data = {col: [f"{col}_{i}" for i in range(num_rows)] for col in columns}
    return pd.DataFrame(data)


# Unit Testing Section
# validate that the ingestion functions/methods and the resulting
# DataFrame structures meet our expectations.

class TestIngestionProductionFileStructure(unittest.TestCase):
    def setUp(self):
        # Define the expected structure for a production Excel file as a list of tuples.
        # Each tuple contains: (sheet_name, number of rows, list of expected column names)
        sheet_specs = [
            ("ProcessorTypeTbl", 15, ["ProcessorTypeIndex", "ProcessorTypeID", "ProcessorType"]),
            ("LocationTbl", 5, ["LocationIndex", "LocationID", "LocationName", "AreaDesc", "SQFT"]),
            ("TaskTbl", 80, ["TaskIndex", "TaskID", "TasksDesc", "ProcessorIndex", "ProcessorID",
                             "ProcessorDesc", "GenRateProcessTime", "StndrdRateProcessTime",
                             "VarRateProcessTime", "LearnCurvePct", "LearnCurveMaxThreshold",
                             "MinProcessTimePct", "MaxProcessTimePct", "YeildPct",
                             "BatchSizeReq", "UnbatchingSize", "ComponentsGenerated"]),
            ("TaskResourcesTbl", 80,
             ["TaskIndex", "TaskID", "TasksDesc"] + [f"NResource{i:02d}" for i in range(1, 51)]),
            ("ResourceTbl", 29, ["ResourceIndex", "ResourceIndex.1", "ResourceName", "ResourceType",
                                 "TotalResourceUnitsAvailable", "DownProbabilityPerUnit",
                                 "DownReplacementDelay", "Units"]),
            ("ProcessorTbl", 11, ["ProcessorIndex", "ProcessorID", "ProcessorDesc", "ProcessorTypeIndex",
                                  "ProcessorTypeID", "ProcessorType", "NumberStations",
                                  "LocationIndex", "LocationID", "Location",
                                  "DownProabilityPerUnity", "DownDelay", "ProcessingUnits"]),
            ("StationTbl", 31, ["StationIndex", "StationID", "StationDesc", "StationType",
                                "StationPriorityMethod", "StationCapacity", "ProcessorIndex",
                                "ProcessorID", "ProcessorDesc", "DownProabilityPerUnit",
                                "DownDelay", "Units"]),
            ("ProductsTbl", 1, ["ProductIndex", "ProductID", "ProductDesc"]),
            ("ConfigTbl", 1, ["ConfigIndex", "ConfigID", "ConfigDesc", "ProductIndex",
                              "ProductID", "ProductDesc"]),
            ("ProcessUnitTbl", 1, ["ProcessUnitIndex", "ProcessUnitID", "ProcessUnitDesc",
                                   "ConfigIndex", "ConfigID", "ConfigDesc", "ProductIndex",
                                   "ProductID", "ProductDesc"]),
            ("ArrivalUnitTbl", 1, ["ArrivalUnitIndex", "ArrivalUnitID", "ArrivalUnitDesc",
                                   "ConfigIndex", "ConfigID", "ConfigDesc", "ProductIndex",
                                   "ProductID", "ProductDesc"]),
            ("ComponentsTbl", 2, ["ComponentIndex", "ComponentID", "ComponentDesc", "UnitType",
                                  "UnitIndex", "UnitID", "UnitDesc", "ConfigIndex",
                                  "ConfigID", "ConfigDesc", "ProductIndex", "ProductID",
                                  "ProductDesc"]),
            ("Product01", 80, ["TaskPred"] + [f"Task I{i}" for i in range(1, 81)]),
            ("ArrivalDatesTbl", 1, ["ArrivalDatesIndex", "ArrivalDatesID", "ArrivalDatesDesc",
                                    "ArrivalDate", "ArrivalCount", "ArrivalUnitIndex",
                                    "ArrivalUnitID", "ArrivalUnitDesc", "ProductIndex",
                                    "ProductID", "ProductDesc"]),
            ("ArrivalRatesTbl", 1, ["ArrivalRatesIndex", "ArrivalRatesID", "ArrivalRatesDesc",
                                    "ArrivalValue", "ArrivalUnits", "ArrivalCount",
                                    "ArrivalUnitIndex", "ArrivalUnitID", "ArrivalUnitDesc",
                                    "ProductIndex", "ProductID", "ProductDesc"]),
            ("DeliveryRatesTbl", 1, ["DeliveryRatesIndex", "DeliveryRatesID", "DeliveryRatesDesc",
                                     "DeliveryRateValue", "DeliveryUnits", "DeliveryCount",
                                     "ProductIndex", "ProductID", "ProductDesc"]),
            ("DeliveryDatesTbl", 1, ["DeliveryRatesIndex", "DeliveryRatesID", "DeliveryRatesDesc",
                                     "DeliveryRateDate", "DeliveryCount", "ProductIndex",
                                     "ProductID", "ProductDesc"]),
            ("PersonnelTbl", 15, ["PersonnelIndex", "PersonnelID", "PersonnelDesc", "Npersonnel",
                                  "ProductionRate", "HourlyRate", "Capacity", "Schedual",
                                  "Shift", "HolidayCalendar", "HolidayDays", "PTOUtilization",
                                  "PTORatePerWeek", "PTOCalendar", "OnTimeStartReliability",
                                  "OnTimeStartFailureDelay", "OvertimeAllowance", "OvertimeRate"]),
            ("SchdPtrnTbl", 3, ["SchdPtrnIndex", "SchdPtrnID", "SchdPtrnDesc", "ShiftStartDate",
                                "Day01", "Day02", "Day03", "Day04", "Day05", "Day06", "Day07"]),
            ("ShiftTbl", 4, ["ShiftIndex", "ShiftID", "ShiftDesc", "ShiftStart", "ShiftEnd",
                              "ShiftStartUnit", "ShiftEndUnit", "ShiftLength", "ShiftBreakStart",
                              "ShiftBreakEnd", "ShiftBreakStartUnit", "ShiftBreakEndUnit",
                              "BreakLength", "StartBuffer", "EndBuffer", "ProductionStartTimePart1",
                              "ProductionEndTimePart1", "ProductionStartTimePart2", "ProductionEndTimePart2",
                              "TotalStaffingTime", "BaseProductionTime"]),
            ("HolidayClndrTbl", 60, ["Year01", "HolidayCalendar01", "HolidayCalendarDay01"]),
            ("PTOTbl", 180, ["PTOYear01", "PTOCalendar01", "PTODay01", "PTODay01.1", "PTOYearlySum01"]),
        ]

        # Use an OrderedDict to maintain the sheet order.
        self.sheets = OrderedDict()
        for sheet_name, num_rows, columns in sheet_specs:
            self.sheets[sheet_name] = create_dummy_df(num_rows, columns)

        # Write the dummy DataFrames to a single Excel file stored in a buffer we will call 'BytesIO'.
        # This simulates the production Excel file for testing the ingestion process.
        self.excel_buffer = io.BytesIO()
        with pd.ExcelWriter(self.excel_buffer, engine="xlsxwriter") as writer:
            for sheet_name, df in self.sheets.items():
                df.to_excel(writer, sheet_name=sheet_name, index=False)
        self.excel_buffer.seek(0)  # Reset to the beginning.

        # Define a dummy dtype mapping for TaskResourcesTbl.
        # This converts all columns that start with "NResource" to type float.
        nresource_cols = [col for col in self.sheets["TaskResourcesTbl"].columns if col.startswith("NResource")]
        self.dtype_mapping = {
            "TaskResourcesTbl": {col: float for col in nresource_cols}
        }


# Test class for a simple 2-sheet Excel file ingestion.
class TestIngestion(unittest.TestCase):
    def setUp(self):
        # Create sample DataFrames to test the ingestion function with a small Excel file.
        self.df_sheet1 = pd.DataFrame({
            "A": [1, 2, 3],
            "B": ["x", "y", "z"]
        })
        self.df_sheet2 = pd.DataFrame({
            "X": [10, 20],
            "Y": ["name1", "name1"]
        })

        # Write these DataFrames to an Excel file in a BytesIO buffer.
        self.excel_buffer = io.BytesIO()
        with pd.ExcelWriter(self.excel_buffer, engine="xlsxwriter") as writer:
            self.df_sheet1.to_excel(writer, sheet_name="Sheet1", index=False)
            self.df_sheet2.to_excel(writer, sheet_name="Sheet2", index=False)
        self.excel_buffer.seek(0)  # Reset the pointer for reading.

        # Define a simple dtype mapping for Sheet2 (ensuring column 'X' is float).
        self.dtype_mapping = {
            "Sheet2": {"X": float}
        }

    def test_ingest_excel_to_dfs(self):
        # Test that the ingestion function correctly reads the Excel file.
        dfs = ingest_excel_to_dfs(self.excel_buffer, dtype_mapping=self.dtype_mapping)

        # Verify that both sheets are present in the returned dictionary.
        self.assertIn("Sheet1", dfs)
        self.assertIn("Sheet2", dfs)

        # Check that the contents of Sheet1 match the original DataFrame.
        pd.testing.assert_frame_equal(dfs["Sheet1"].reset_index(drop=True), self.df_sheet1)

        # For Sheet2, ensure that column 'X' has been converted to float.
        self.assertTrue(pd.api.types.is_float_dtype(dfs["Sheet2"]["X"]))
        expected_sheet2 = self.df_sheet2.astype({"X": float})
        pd.testing.assert_frame_equal(dfs["Sheet2"].reset_index(drop=True), expected_sheet2)

    def test_create_table_mapping(self):
        # Test the mapping function that summarizes the ingested DataFrames.
        dfs = ingest_excel_to_dfs(self.excel_buffer, dtype_mapping=self.dtype_mapping)
        mapping_df = create_table_mapping(dfs)

        # Ensure the mapping DataFrame has the expected columns.
        for col in ["table_name", "num_columns", "num_rows", "columns"]:
            self.assertIn(col, mapping_df.columns)

        # There should be exactly two entries corresponding to the two sheets.
        self.assertEqual(len(mapping_df), 2)

        # Validate that the summary for Sheet1 matches its dimensions.
        sheet1_mapping = mapping_df[mapping_df["table_name"] == "Sheet1"].iloc[0]
        self.assertEqual(sheet1_mapping["num_columns"], self.df_sheet1.shape[1])
        self.assertEqual(sheet1_mapping["num_rows"], self.df_sheet1.shape[0])

        # Validate that the summary for Sheet2 matches its dimensions.
        sheet2_mapping = mapping_df[mapping_df["table_name"] == "Sheet2"].iloc[0]
        self.assertEqual(sheet2_mapping["num_columns"], self.df_sheet2.shape[1])
        self.assertEqual(sheet2_mapping["num_rows"], self.df_sheet2.shape[0])


# Function: ingest_excel_to_dfs I am ingesting the files create for testing. Pretty straitforward
# Reads an Excel file and returns a dictionary mapping sheet names to DataFrames.
# If a dtype_mapping is provided, columns in each sheet are cast to the specified types.
# Note: make this a doc string
def ingest_excel_to_dfs(file_path_or_buffer, dtype_mapping: dict = None) -> dict:
    excel_file = pd.ExcelFile(file_path_or_buffer)
    dfs = {}
    # Loop through each sheet in the Excel file.
    for sheet in excel_file.sheet_names:
        df = excel_file.parse(sheet)
        # Apply data type conversions if specified for this sheet.
        if dtype_mapping and sheet in dtype_mapping:
            df = df.astype(dtype_mapping[sheet])
        dfs[sheet] = df
    return dfs

# Function: create_table_mapping
# Creates a summary DataFrame that maps the structure of each ingested sheet.
# The summary includes the table name, number of columns, number of rows,
# and a list of column names.
# Note: make this a doc string

def create_table_mapping(dfs: dict) -> pd.DataFrame:
    mapping = []
    for name, df in dfs.items():
        mapping.append({
            "table_name": name,
            "num_columns": df.shape[1],
            "num_rows": df.shape[0],
            "columns": list(df.columns)
        })
    return pd.DataFrame(mapping)


# Run Unit Tests
# -----------------------------------------------------------------------------
# runs all defined unit tests to verify that the ingestion and mapping
# functions work as expected.
unittest.main(argv=['first-arg-is-ignored'], exit=False)


In [ ]:
# Install the required packages for Excel writing and discrete-event simulation. The Version numbers have not been solidified att.
!pip install XlsxWriter simpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.1/165.1 kB 3.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import io
import pandas as pd
import tempfile
import os
from google.colab import files

In [ ]:
def ingest_excel_to_dfs(file_path_or_buffer, dtype_mapping: dict = None) -> dict:
    """
    Reads an Excel file and returns a dictionary mapping each sheet name to a pandas DataFrame.
    If a dtype_mapping is provided for a sheet, it converts the columns to the specified data types.

    Parameters:
        file_path_or_buffer (str or file-like): Path to the Excel file or a file-like object.
        dtype_mapping (dict): A mapping of sheet names to a dict of column names and their desired types.

    Returns:
        dict: A dictionary where keys are sheet names and values are DataFrames.
    """
    excel_file = pd.ExcelFile(file_path_or_buffer)
    dfs = {}
    for sheet in excel_file.sheet_names:
        df = excel_file.parse(sheet)
        # If there are calculated values, they will be read in as the last computed value.
        # Apply the data type conversion if a mapping is provided for this sheet.
        if dtype_mapping and sheet in dtype_mapping:
            try:
                df = df.astype(dtype_mapping[sheet])
            except Exception as e:
                print(f"Warning: Could not convert types for sheet '{sheet}': {e}")
        dfs[sheet] = df
    return dfs

def create_table_mapping(dfs: dict) -> pd.DataFrame:
    """
    Creates a summary mapping DataFrame of the ingested tables.

    Each row in the mapping contains:
        - table_name: Name of the sheet/table.
        - num_columns: Number of columns in the table.
        - num_rows: Number of rows in the table.
        - columns: List of column names.

    Parameters:
        dfs (dict): A dictionary with sheet names as keys and DataFrames as values.

    Returns:
        pd.DataFrame: A DataFrame summarizing the ingested tables.
    """
    mapping = []
    for name, df in dfs.items():
        mapping.append({
            "table_name": name,
            "num_columns": df.shape[1],
            "num_rows": df.shape[0],
            "columns": list(df.columns)
        })
    return pd.DataFrame(mapping)


In [ ]:
# Ingestion and Printing in Colab
# This block prompts the user to upload an Excel file and then processes each
# uploaded file. The file is expected to have multiple sheets, each representing a
# table with specific production system data. Use the intake form instuctions to
# learn how to fill out the intake form
uploaded = files.upload()

# For each uploaded file, we define sheet names and how they are mapped to their normal data types.
# This "dtype_map" ensures that each column is correctly set/mapped/related to the desired data type.
for fn in uploaded.keys():
    dtype_map = {
        'ProcessorTypeTbl': {
            'ProcessorTypeIndex': int,
            'ProcessorTypeID': str,
            'ProcessorType': str,
        },
        'LocationTbl': {
            'LocationIndex': int,
            'LocationID': str,
            'LocationName': str,
            'AreaDesc': str,
            'SQFT': int,
        },
        'TaskTbl': {
            'TaskIndex': int,
            'TaskID': str,
            'TasksDesc': str,
            'ProcessorIndex': int,
            'ProcessorID': str,
            'ProcessorDesc': str,
            'GenRateProcessTime': float,
            'StndrdRateProcessTime': float,
            'VarRateProcessTime': float,
            'LearnCurvePct': float,
            'LearnCurveMaxThreshold': float,
            'MinProcessTimePct': float,
            'MaxProcessTimePct': float,
            'YeildPct': float,
            'BatchSizeReq': float,
            'UnbatchingSize': float,
            'ComponentsGenerated': str,
        },
        # For TaskResourcesTbl, we initially read all columns as strings;
        # later, columns with default names will be renamed to a standard format.
        'TaskResourcesTbl': {col: str for col in []},
        'ResourceTbl': {
            'ResourceIndex': int,
            'ResourceIndex.1': str,
            'ResourceName': str,
            'ResourceType': str,
            'TotalResourceUnitsAvailable': int,
            'DownProbabilityPerUnit': int,
            'DownReplacementDelay': int,
            'Units': str,
        },
        'ProcessorTbl': {
            'ProcessorIndex': int,
            'ProcessorID': str,
            'ProcessorDesc': str,
            'ProcessorTypeIndex': int,
            'ProcessorTypeID': str,
            'ProcessorType': str,
            'NumberStations': int,
            'LocationIndex': int,
            'LocationID': str,
            'Location': str,
            'DownProabilityPerUnity': int,
            'DownDelay': int,
            'ProcessingUnits': str,
        },
        'StationTbl': {
            'StationIndex': int,
            'StationID': str,
            'StationDesc': str,
            'StationType': str,
            'StationPriorityMethod': str,
            'StationCapacity': int,
            'ProcessorIndex': int,
            'ProcessorID': str,
            'ProcessorDesc': str,
            'DownProabilityPerUnit': int,
            'DownDelay': int,
            'Units': str,
        },
        'ProductsTbl': {
            'ProductIndex': int,
            'ProductID': str,
            'ProductDesc': str,
        },
        'ConfigTbl': {
            'ConfigIndex': int,
            'ConfigID': str,
            'ConfigDesc': str,
            'ProductIndex': int,
            'ProductID': str,
            'ProductDesc': str,
        },
        'ProcessUnitTbl': {
            'ProcessUnitIndex': int,
            'ProcessUnitID': str,
            'ProcessUnitDesc': str,
            'ConfigIndex': int,
            'ConfigID': str,
            'ConfigDesc': str,
            'ProductIndex': int,
            'ProductID': str,
            'ProductDesc': str,
        },
        'ArrivalUnitTbl': {
            'ArrivalUnitIndex': int,
            'ArrivalUnitID': str,
            'ArrivalUnitDesc': str,
            'ConfigIndex': int,
            'ConfigID': str,
            'ConfigDesc': str,
            'ProductIndex': int,
            'ProductID': str,
            'ProductDesc': str,
        },
        'ComponentsTbl': {
            'ComponentIndex': int,
            'ComponentID': str,
            'ComponentDesc': str,
            'UnitType': str,
            'UnitIndex': int,
            'UnitID': str,
            'UnitDesc': str,
            'ConfigIndex': int,
            'ConfigID': str,
            'ConfigDesc': str,
            'ProductIndex': int,
            'ProductID': str,
            'ProductDesc': str,
        },
        # For Product01, we later rename task columns to a more user-friendly format.
        'Product01': {
            'TaskPred': str,
            # The task columns will be renamed post ingestion.
        },
        'ArrivalDatesTbl': {
            'ArrivalDatesIndex': int,
            'ArrivalDatesID': str,
            'ArrivalDatesDesc': str,
            'ArrivalDate': 'datetime64[ns]',
            'ArrivalCount': int,
            'ArrivalUnitIndex': int,
            'ArrivalUnitID': str,
            'ArrivalUnitDesc': str,
            'ProductIndex': int,
            'ProductID': str,
            'ProductDesc': str,
        },
        'ArrivalRatesTbl': {
            'ArrivalRatesIndex': int,
            'ArrivalRatesID': str,
            'ArrivalRatesDesc': str,
            'ArrivalValue': int,
            'ArrivalUnits': str,
            'ArrivalCount': int,
            'ArrivalUnitIndex': int,
            'ArrivalUnitID': str,
            'ArrivalUnitDesc': str,
            'ProductIndex': int,
            'ProductID': str,
            'ProductDesc': str,
        },
        'DeliveryRatesTbl': {
            'DeliveryRatesIndex': int,
            'DeliveryRatesID': str,
            'DeliveryRatesDesc': str,
            'DeliveryRateValue': int,
            'DeliveryUnits': str,
            'DeliveryCount': int,
            'ProductIndex': int,
            'ProductID': str,
            'ProductDesc': str,
        },
        'DeliveryDatesTbl': {
            'DeliveryRatesIndex': int,
            'DeliveryRatesID': str,
            'DeliveryRatesDesc': str,
            'DeliveryRateDate': 'datetime64[ns]',
            'DeliveryCount': int,
            'ProductIndex': int,
            'ProductID': str,
            'ProductDesc': str,
        },
        'PersonnelTbl': {
            'PersonnelIndex': int,
            'PersonnelID': str,
            'PersonnelDesc': str,
            'Npersonnel': int,
            'ProductionRate': int,
            'HourlyRate': int,
            'Capacity': int,
            'Schedual': str,
            'Shift': str,
            'HolidayCalendar': str,
            'HolidayDays': str,
            'PTOUtilization': float,
            'PTORatePerWeek': float,
            'PTOCalendar': str,
            'OnTimeStartReliability': int,
            'OnTimeStartFailureDelay': int,
            'OvertimeAllowance': int,
            'OvertimeRate': int,
        },
        'SchdPtrnTbl': {
            'SchdPtrnIndex': int,
            'SchdPtrnID': str,
            'SchdPtrnDesc': str,
            'ShiftStartDate': 'datetime64[ns]',
            'Day01': str,
            'Day02': str,
            'Day03': str,
            'Day04': str,
            'Day05': str,
            'Day06': str,
            'Day07': str,
        },
        'ShiftTbl': {
            'ShiftIndex': int,
            'ShiftID': str,
            'ShiftDesc': str,
            'ShiftStart': str,
            'ShiftEnd': str,
            'ShiftStartUnit': int,
            'ShiftEndUnit': int,
            'ShiftLength': int,
            'ShiftBreakStart': str,
            'ShiftBreakEnd': str,
            'ShiftBreakStartUnit': int,
            'ShiftBreakEndUnit': int,
            'BreakLength': int,
            'StartBuffer': int,
            'EndBuffer': int,
            'ProductionStartTimePart1': int,
            'ProductionEndTimePart1': int,
            'ProductionStartTimePart2': int,
            'ProductionEndTimePart2': int,
            'TotalStaffingTime': int,
            'BaseProductionTime': int,
        },
        'HolidayClndrTbl': {
            'Year01': int,
            'HolidayCalendar01': 'datetime64[ns]',
            'HolidayCalendarDay01': int,
        },
        'PTOTbl': {
            'PTOYear01': int,
            'PTOCalendar01': 'datetime64[ns]',
            'PTODay01': int,
            'PTODay01.1': int,
            'PTOYearlySum01': int,
        },
    }

    # Ingest the Excel file into a dictionary of DataFrames.
    # The function "ingest_excel_to_dfs" reads each sheet, applies the data type mapping,
    # and returns a dictionary with sheet names as keys.
    dfs = ingest_excel_to_dfs(fn, dtype_mapping=dtype_map)

    # Post-Ingestion Renaming Steps
    # These steps clean up default column names that arise during ingestion,
    # making the DataFrames more user-friendly and standardized.

    # For TaskResourcesTbl, rename any columns with default names (e.g., "Unnamed: ...")
    # to a standardized format "NResourceXX" (where XX is a two-digit number).
    if 'TaskResourcesTbl' in dfs:
        tr_df = dfs['TaskResourcesTbl']
        rename_dict = {}
        for col in tr_df.columns:
            if col.startswith("Unnamed:"):
                try:
                    num = int(col.split(":")[1].strip())
                except ValueError:
                    num = 0
                rename_dict[col] = f"NResource{num:02d}"
        tr_df.rename(columns=rename_dict, inplace=True)

    # For Product01, rename task columns to a clearer format.
    # For example, change "task001" to "Task I1", "task002" to "Task I2", etc.
    if 'Product01' in dfs:
        p1_df = dfs['Product01']
        rename_dict = {}
        for col in p1_df.columns:
            # Exclude the 'TaskPred' column from renaming.
            if col.lower().startswith("task") and col.lower() != "taskpred":
                try:
                    num = int(col[4:])  # Extract the numeric portion after "task"
                except ValueError:
                    continue
                rename_dict[col] = f"Task I{num}"
        p1_df.rename(columns=rename_dict, inplace=True)

    # Create a summary mapping of the ingested DataFrames using the helper function.
    # This mapping includes the sheet name, number of columns, number of rows, and the column names.
    mapping_df = create_table_mapping(dfs)

    # Print the high-level summary mapping of all tables.
    print("TABLE MAPPING SUMMARY:")
    print(mapping_df)

    # For each table, print detailed information (DataFrame info and descriptive statistics)
    # to help the user verify that data ingestion was successful and correct.
    for table_name, df in dfs.items():
        print("\n" + "="*50)
        print(f"Table: {table_name}")
        print("="*50)
        print("DataFrame Info:")
        df.info()
        print("\nDataFrame Summary (describe):")
        print(df.describe(include='all'))



Saving production_system_intake_Example01.xlsx to production_system_intake_Example01.xlsx
TABLE MAPPING SUMMARY:
          table_name  num_columns  num_rows  \
0   ProcessorTypeTbl            3        15   
1        LocationTbl            5         5   
2            TaskTbl           17        80   
3   TaskResourcesTbl           53        80   
4        ResourceTbl            8        29   
5       ProcessorTbl           13        11   
6         StationTbl           12        31   
7        ProductsTbl            3         1   
8          ConfigTbl            6         1   
9     ProcessUnitTbl            9         1   
10    ArrivalUnitTbl            9         1   
11     ComponentsTbl           13         2   
12         Product01           81        80   
13   ArrivalDatesTbl           11         1   
14   ArrivalRatesTbl           12         1   
15  DeliveryRatesTbl            9         1   
16  DeliveryDatesTbl            8         1   
17      PersonnelTbl           18        